#구글드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/gdrive/',force_remount = True)

Mounted at /gdrive/


#라이브러리들

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from numba import jit
import os
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

## 기본 데이터 준비

In [ ]:
gp=pd.read_csv('/gdrive/MyDrive/프로젝트2/구매상품TR_reorder.csv',chunksize = 2000000)
gp_li = list(gp)
order = pd.concat(gp_li)

customer = pd.read_csv('/gdrive/MyDrive/프로젝트2/cust.csv')

goods =  pd.read_csv('/gdrive/MyDrive/프로젝트2/goods.csv')

# Class

In [ ]:
class project():
  def __init__(self,customer, goods, order):
    self.goods = goods
    self.customer = customer
    self.order = order
#-------------------------데이터프레임 ---------------------------------------------------------------------------
  def show_order(self):
    return self.order
  def show_customer(self):
    return self.customer
  def show_goods(self):
    return self.goods
#-------------------------  RFM ----------------------------------------------------------------------------
  def get_rfm_14_binary(self):
    temp = self.customer[['14_4분기_구매금액','14년도_영수증갯수','14년도_구매금액']].copy()
    temp.columns = ['recency','frequency','monetary']
    print(np.median(temp.frequency))
    print(np.median(temp.monetary))
    temp.recency = np.where(temp.recency != 0, 1,0)
    temp.frequency = np.where(temp.frequency < np.median(temp.frequency),0,1)
    temp.monetary = np.where(temp.monetary < np.median(temp.monetary),0,1)
    li = []
    for i in temp.values.tolist():
      if i not in li:
        li.append(i)
    res=self.get_cust_seg(temp,sorted(li,reverse = True))
    temp['cust_seg'] =res
    return temp
  def get_rfm_15_binary(self):
    temp = self.customer[['15_4분기_구매금액','15년도_영수증갯수','15년도_구매금액']].copy()
    temp.columns = ['recency','frequency','monetary']
    print(np.median(temp.frequency))
    print(np.median(temp.monetary))
    temp.recency = np.where(temp.recency != 0, 1,0)
    temp.frequency = np.where(temp.frequency < np.median(temp.frequency),0,1)
    temp.monetary = np.where(temp.monetary < np.median(temp.monetary),0,1)
    print(np.median(temp.frequency))
    li = []
    for i in temp.values.tolist():
      if i not in li:
        li.append(i)
    res=self.get_cust_seg(temp,sorted(li,reverse = True))
    temp['cust_seg'] =res
    return temp

  def get_rfm_14(self):
    temp = self.customer[['14_4분기_구매금액','14년도_영수증갯수','14년도_구매금액']].copy()
    temp.columns = ['recency','frequency','monetary']
    temp.recency = np.where(temp.recency != 0, 1,0)
    return temp

  def get_rfm_15(self):
    temp = self.customer[['15_4분기_구매금액','15년도_영수증갯수','15년도_구매금액']].copy()
    temp.columns = ['recency','frequency','monetary']
    temp.recency = np.where(temp.recency != 0, 1,0)
    return temp
  def get_cust_seg(self,temp,li):
    res = []
    for x in range(len(temp)):
      res.append(li.index(temp.iloc[x].values.tolist()))
    return res
#------------------------군집화 및 주성분분석 시각화 -----------------------------------------------------------
  def get_kmeans_clustering(self,rfm,n_cluster):
    scaler = MinMaxScaler()
    scaled_value=scaler.fit_transform(rfm.values)
    kmeans = KMeans(n_clusters = n_cluster)
    kmeans.fit(scaled_value)
    rfm['cluster']=kmeans.labels_
    return rfm
  def get_cluster_scatterplot(self,dataframe):
    pca = PCA(n_components = 2)
    temp=pca.fit_transform(dataframe.iloc[:,:-1])
    res=pd.DataFrame(temp)

    res['cluster']=dataframe.iloc[:,-1].values.tolist()
    sns.scatterplot(data = res,x = res.columns[0],y = res.columns[1],hue = res.columns[-1])
    plt.show()
    return pca.explained_variance_ratio_
#--------------------------------------------------------------------------------------------------
  def set_feature_customer(self,featurename,values):
    self.customer[featurename] = values
  def merge_goods_order(self):
    return self.order
  def train(self):
    df = self.merge_goods_order()
    return df[df['구매일자']<20151000]
  def test(self):
    df = self.merge_goods_order()
    return df[df['구매일자']>20151000]
  def get_shopping_list_middle(self,goods_order):
    li = []
    for x in goods_order['영수증번호'].unique():
      li.append(goods_order[goods_order['영수증번호']==x]['Unnamed: 5'].unique().tolist())
    return li
  @jit(nopython = False)
  def get_receipt_order(self,number,o):
    res=o[o['고객번호']== number]['영수증번호'].unique()
    return res
  @jit(nopython = False)
  def get_goods_name(self,receipt_number,o):
    res=o[o['영수증번호']== receipt_number]['소분류코드'].values
    return res
  
  @jit(nopython = False)
  def get_add_to_cart_order(self,array1,array2):
    sum = []
    for x in range(len(array1)):
      sum += [i for i in range(1,array2[x]+1)]
    return sum
  def get_customer_id_list(self):
    return sorted(self.customer['고객번호'].unique())
  @jit(nopython = False)
  def comparison(self,current,past):
    res = []
    if len(current) == 1:
      if current in past:
        res += [1]
      else :
        res += [0]
    else :
      res+=(np.in1d(current,past)*1).tolist()
    return res
  @jit(nopython = False)
  def cal_reordered(self):
    d = self.merge_goods_order()
    o= d[['고객번호','영수증번호','소분류코드']].copy()
    cust_list= self.get_customer_id_list()
    res = []
    for cust_id in cust_list[1000:2000]:
      print(cust_id)
      length=self.get_receipt_order(cust_id,o)
      for i in range(len(length)):
        if i != 0:
          past=self.get_goods_name(length[i-1],o)
          current=self.get_goods_name(length[i],o)
          res+=self.comparison(current,past)
        else :
          current=self.get_goods_name(length[i],o)
          for i in range(len(current)):
            res +=[0]
    pd.DataFrame(res).to_csv(f'/gdrive/MyDrive/프로젝트2/{cust_id}.csv',index = False)
    return res
  @jit(nopython = False)
  def get_cust_goods_kinds(self,dataframe):
    o=dataframe
    d = o.groupby(['고객번호'])['소분류코드'].value_counts().reset_index(name = 'cnt')
    res = []
    print(d)
    cust_list = self.get_customer_id_list()
    for x in cust_list:
      res.append(len(d[d['고객번호']== x]['소분류코드'].unique()))
    return res
  @jit(nopython = False)
  def get_cust_goods_kinds_mean(self,dataframe):
    o=dataframe
    d = o.groupby(['고객번호','영수증번호'])['소분류코드'].count().reset_index(name = 'cnt')
    res = []
    print(d)
    cust_list = self.get_customer_id_list()
    for x in cust_list:
      res.append(d[d['고객번호'] == x]['cnt'].mean())
    return res        

##Class 생성

In [ ]:
p=project(customer,goods,order)

# 필요 데이터셋 생성

##항목수와 평균

In [ ]:
dataframe = p.train()
res1=p.get_cust_goods_kinds(dataframe)

In [ ]:
res2=p.get_cust_goods_kinds_mean(dataframe)

<ipython-input-14-95b5d48242c6>:154: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "get_cust_goods_kinds_mean" failed type inference due to: non-precise type pyobject
During: typing of argument at <ipython-input-14-95b5d48242c6> (156)

File "<ipython-input-14-95b5d48242c6>", line 156:
  def get_cust_goods_kinds_mean(self,dataframe):
    o=dataframe
    ^

  @jit(nopython = False)
<ipython-input-14-95b5d48242c6>:154: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "get_cust_goods_kinds_mean" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-14-95b5d48242c6>", line 161:
  def get_cust_goods_kinds_mean(self,dataframe):
      <source elided>
    cust_list = self.get_customer_id_list()
    for x in cust_list:
    ^

  @jit(nopython = False)
/usr/local/lib/python3.8/dist-packages/numba/core/object_mode_passes.

          고객번호     영수증번호  cnt
0            1     93124    1
1            1    100083    3
2            1    107062    1
3            1    113352    1
4            1    114242    1
...        ...       ...  ...
9690877  19383   5162133    1
9690878  19383   9832756    8
9690879  19383  10047102    1
9690880  19383  10047211    2
9690881  19383  10355251    5

[9690882 rows x 3 columns]


<ipython-input-14-95b5d48242c6>:154: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "get_cust_goods_kinds_mean" failed type inference due to: non-precise type pyobject
During: typing of argument at <ipython-input-14-95b5d48242c6> (161)

File "<ipython-input-14-95b5d48242c6>", line 161:
  def get_cust_goods_kinds_mean(self,dataframe):
      <source elided>
    cust_list = self.get_customer_id_list()
    for x in cust_list:
    ^

  @jit(nopython = False)
/usr/local/lib/python3.8/dist-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "get_cust_goods_kinds_mean" was compiled in object mode without forceobj=True.

File "<ipython-input-14-95b5d48242c6>", line 161:
  def get_cust_goods_kinds_mean(self,dataframe):
      <source elided>
    cust_list = self.get_customer_id_list()
    for x in cust_list:
    ^

  warnings.warn(errors.NumbaWarning(warn_msg,
/usr/local/lib/python3.8/dist-packages/numba/core/object_mode_pa

##Reorder

In [ ]:
res = p.cal_reordered()

<ipython-input-7-e3c8edbe8301>:124: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "cal_reordered" failed type inference due to: non-precise type pyobject
During: typing of argument at <ipython-input-7-e3c8edbe8301> (126)

File "<ipython-input-7-e3c8edbe8301>", line 126:
  def cal_reordered(self):
    d = self.merge_goods_order()
    ^

  @jit(nopython = False)
<ipython-input-7-e3c8edbe8301>:124: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "cal_reordered" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-7-e3c8edbe8301>", line 130:
  def cal_reordered(self):
      <source elided>
    res = []
    for cust_id in cust_list[1000:2000]:
    ^

  @jit(nopython = False)
/usr/local/lib/python3.8/dist-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "cal_reordered" was compiled in object 

1001


/usr/local/lib/python3.8/dist-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "get_goods_name" was compiled in object mode without forceobj=True.

File "<ipython-input-7-e3c8edbe8301>", line 101:
  @jit(nopython = False)
  def get_goods_name(self,receipt_number,o):
  ^

  warnings.warn(errors.NumbaWarning(warn_msg,
/usr/local/lib/python3.8/dist-packages/numba/core/object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit

File "<ipython-input-7-e3c8edbe8301>", line 101:
  @jit(nopython = False)
  def get_goods_name(self,receipt_number,o):
  ^

  warnings.warn(errors.NumbaDeprecationWarning(msg,
<ipython-input-7-e3c8edbe8301>:113: NumbaWarning: 
Compilation is falling back to object mode W

1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1161
1162
1163
1164
1165
1166
1167
1168
1169
1170
1171
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1192
1193
1194
1195
1196
1197
1198
1199
1200
1201


In [ ]:
import os
os.listdir('/gdrive/MyDrive/프로젝트2/프로젝트')[-1]

'각고객의구매당평균항목수.csv'

##rfm

In [ ]:
df=p.get_rfm_15()
data=p.get_kmeans_clustering(df,8)

In [ ]:
df_new = df.copy()
df_new['cluster'] = data['cluster']
p.get_cluster_scatterplot(df_new)

In [ ]:
df_bin=p.get_rfm_15_binary()
df_bin.to_csv('15rfm.csv',index = False)

In [ ]:
p.get_rfm_14_binary()

270.0
8357765.0


,recency,frequency,monetary,cust_seg
0,1,1,1,0
1,1,1,1,0
2,1,1,0,1
3,1,0,1,2
4,1,1,0,1
...,...,...,...,...
19378,1,0,0,3
19379,1,0,0,3
19380,0,0,0,5
19381,0,0,0,5


##구매감소예측

In [ ]:
p.show_customer()['반기별_고객분류']

In [ ]:
p.set_feature_customer('구매증감지수',p.show_customer()['15_4분기_구매금액']/p.show_customer()['14_1분기_구매금액'])

In [ ]:
cus=p.show_customer()
sum=(cus['15년도_구매금액'].sum()/cus['14년도_구매금액'].sum())
d=cus[cus['구매증감지수'] < sum].groupby(['성별','연령대'])[['총구매금액']].mean().reset_index()

In [ ]:
d

In [ ]:
sns.barplot(data = d,x = '연령대',y = '총구매금액',hue = '성별')

In [ ]:
cus['15년도_구매금액'].sum()

347417316360

In [ ]:
cus['14년도_구매금액'].sum()

329601840581

In [ ]:
new = cus[['14_1분기_구매금액','14_2분기_구매금액', '14_3분기_구매금액', '14_4분기_구매금액', '15_1분기_구매금액','15_2분기_구매금액', '15_3분기_구매금액', '15_4분기_구매금액']]
#계절성제거해야됨


target=((new['15_3분기_구매금액']-new['14_1분기_구매금액'])/new['14_1분기_구매금액'])*100
sum=((new['15_3분기_구매금액'].sum()-new['14_1분기_구매금액'].sum())/new['14_1분기_구매금액'].sum())*100

target2=((new['15_4분기_구매금액']-new['14_1분기_구매금액'])/new['14_1분기_구매금액'])*100
sum2=((new['15_4분기_구매금액'].sum()-new['14_1분기_구매금액'].sum())/new['14_1분기_구매금액'].sum())*100

In [ ]:
target=np.where(target == np.inf,0,target)
target2 = np.where(target2==np.inf,0,target2)

In [ ]:
cus['target_3'] = target.tolist()
cus['target'] = target2.tolist()

In [ ]:
cus['label_3'] = np.where(cus.target_3 < sum,0,1)
cus['label'] = np.where(cus.target < sum2,0,1)

In [ ]:
len(cus[cus['label'] == 0])/len(cus)

0.5857194448743744

In [ ]:
encoder = LabelEncoder()
sex=encoder.fit_transform(cus.성별)
age = encoder.fit_transform(cus.연령대)
address = encoder.fit_transform(cus.거주지역.astype('object'))

In [ ]:
cus['성별'] = sex
cus['연령대'] = age
cus['거주지역'] = address

In [ ]:
x_train=cus[['성별', '연령대', '거주지역','14_1분기_구매금액',
       '14_2분기_구매금액', '14_3분기_구매금액', '14_4분기_구매금액', '15_1분기_구매금액',
       '15_2분기_구매금액', '15_3분기_구매금액','14년도_1분기_영수증갯수', '14년도_2분기_영수증갯수',
       '14년도_3분기_영수증갯수', '14년도_4분기_영수증갯수','15년도_1분기_영수증갯수.1',
       '15년도_2분기_영수증갯수.1', '15년도_3분기_영수증갯수']]
y_train = cus[['label_3']]

In [ ]:
x_test=cus[['성별', '연령대', '거주지역','14_1분기_구매금액',
       '14_2분기_구매금액', '14_3분기_구매금액', '14_4분기_구매금액', '15_1분기_구매금액',
       '15_2분기_구매금액', '15_3분기_구매금액','14년도_1분기_영수증갯수', '14년도_2분기_영수증갯수',
       '14년도_3분기_영수증갯수', '14년도_4분기_영수증갯수','15년도_1분기_영수증갯수.1',
       '15년도_2분기_영수증갯수.1', '15년도_3분기_영수증갯수']]
y_test = cus[['label']]

In [ ]:

lr = LogisticRegression()
model = LGBMClassifier()
lr.fit(x_train,y_train)
model.fit(x_train,y_train)
preds=lr.predict(x_test)
prds2 = model.predict(x_test)
accuracy_score(preds,y_test)
accuracy_score(prds2,y_test)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7210958056028478

In [ ]:
dataset = [['Milk', 'Onion', 'Nutmeg', 'Eggs', 'Yogurt'],
           ['Onion', 'Nutmeg', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Ice cream', 'Eggs']]

te = TransactionEncoder()
te_ary = te.fit_transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)


frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)

array([   0,    9,   22, 1381,   16,   12,   13,   17,   28, 1264,   63,
          8,   27,   20,   21,   27,   72])

In [ ]:
goods_order=p.merge_goods_order()

In [ ]:
goods_order

,제휴사,영수증번호,대분류코드_x,중분류코드,소분류코드,고객번호,점포코드,구매일자,구매시간,구매금액,대분류코드_y,중분류명,Unnamed: 5,소분류명
0,B,8664000,15,1504,B150401,17218,44,20140222,20,2420,가공식품,탄산음료,음료,사이다
1,B,8664007,15,1504,B150401,11303,44,20140222,13,2400,가공식품,탄산음료,음료,사이다
2,B,8919520,15,1504,B150401,13210,48,20150921,19,5250,가공식품,탄산음료,음료,사이다
3,B,6956422,15,1504,B150401,13823,12,20140813,18,2250,가공식품,탄산음료,음료,사이다
4,B,6957156,15,1504,B150401,18970,12,20140831,22,4900,가공식품,탄산음료,음료,사이다
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28593025,B,7269975,14,1415,B141503,134,16,20141130,15,8900,가공식품,전통과자,과자류,온라인팥빙수재료
28593026,B,7946814,68,6802,B680209,18484,26,20141227,14,1000,패션잡화,여성양말,양말,여성발가락
28593027,B,7777110,7,702,B070203,5043,21,20151012,22,4510,가공식품,즉석구이안주,즉석/편의,즉석쥐포
28593028,B,7579178,7,703,B070305,4073,18,20140125,20,8900,가공식품,마른안주,즉석/편의,온라인 안주


In [ ]:
li = p.get_shopping_list_middle(goods_order)
te = TransactionEncoder()
te_ary = te.fit_transform(li)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)
frequent_itemsets.to_csv('/gdrive/MyDrive/프로젝트2/arule.csv')

SyntaxError: ignored

In [ ]:
frequent_itemsets.to_csv('/gdrive/MyDrive/프로젝트2/프로젝트/연관성.csv',index = False) 

In [ ]:
df_cust=p.show_customer()[['고객번호', '성별', '연령대', '거주지역','14_1분기_구매금액',
       '14_2분기_구매금액', '14_3분기_구매금액', '14_4분기_구매금액', '15_1분기_구매금액',
       '15_2분기_구매금액', '15_3분기_구매금액', '15_4분기_구매금액','14년도_1분기_영수증갯수', '14년도_2분기_영수증갯수',
       '14년도_3분기_영수증갯수', '14년도_4분기_영수증갯수', '15년도_1분기_영수증갯수', '15년도_2분기_영수증갯수',
       '15년도_3분기_영수증갯수', '15년도_4분기_영수증갯수']]

In [ ]:
df_cust['분기별_구매금액_평균']=df_cust[['14_1분기_구매금액',
       '14_2분기_구매금액', '14_3분기_구매금액', '14_4분기_구매금액', '15_1분기_구매금액',
       '15_2분기_구매금액','15_3분기_구매금액']].T.mean()


<ipython-input-17-ad030cb6aa16>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cust['분기별_구매금액_평균']=df_cust[['14_1분기_구매금액',


In [ ]:
df_cust['분기별_영수증갯수_평균']=df_cust[['14년도_1분기_영수증갯수', '14년도_2분기_영수증갯수',
       '14년도_3분기_영수증갯수', '14년도_4분기_영수증갯수', '15년도_1분기_영수증갯수', '15년도_2분기_영수증갯수','15년도_3분기_영수증갯수']].T.mean()

<ipython-input-19-539e98e995d2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cust['분기별_영수증갯수_평균']=df_cust[['14년도_1분기_영수증갯수', '14년도_2분기_영수증갯수',


In [ ]:
df_cust['label'] = np.where(df_cust['15_3분기_구매금액'] != 0,1,0)

<ipython-input-31-617c53bd338b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cust['label'] = np.where(df_cust['15_3분기_구매금액'] != 0,1,0)


In [ ]:
df_cust.columns

Index(['고객번호', '성별', '연령대', '거주지역', '14_1분기_구매금액', '14_2분기_구매금액',
       '14_3분기_구매금액', '14_4분기_구매금액', '15_1분기_구매금액', '15_2분기_구매금액',
       '15_3분기_구매금액', '15_4분기_구매금액', '14년도_1분기_영수증갯수', '14년도_2분기_영수증갯수',
       '14년도_3분기_영수증갯수', '14년도_4분기_영수증갯수', '15년도_1분기_영수증갯수', '15년도_2분기_영수증갯수',
       '15년도_3분기_영수증갯수', '15년도_4분기_영수증갯수', '분기별_구매금액_평균', '분기별_영수증갯수_평균',
       'label'],
      dtype='object')

In [ ]:
dataset = df_cust[['고객번호', '성별', '연령대', '거주지역','분기별_구매금액_평균','분기별_영수증갯수_평균','label']]

In [ ]:
dataset

,고객번호,성별,연령대,거주지역,분기별_구매금액_평균,분기별_영수증갯수_평균,label
0,1,M,60세이상,60.0,1.199001e+07,93.833333,1
1,2,M,60세이상,100.0,1.233949e+07,86.833333,1
2,3,M,60세이상,33.0,4.143415e+05,65.000000,1
3,4,F,60세이상,16.0,2.159426e+06,62.666667,1
4,5,M,60세이상,100.0,1.770135e+06,62.666667,0
...,...,...,...,...,...,...,...
19378,19379,F,20세~24세,69.0,5.977567e+05,35.666667,1
19379,19380,F,20세~24세,39.0,2.267750e+05,11.333333,1
19380,19381,F,40세~44세,460.0,4.661952e+05,25.833333,1
19381,19382,F,45세~49세,460.0,3.044565e+05,24.333333,1


In [ ]:
en = LabelEncoder()
sex=en.fit_transform(dataset['성별'])
age=en.fit_transform(dataset['연령대'])

In [ ]:
dataset['성별'] = sex
dataset['연령대'] = age

<ipython-input-36-80506ea1d8d5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['성별'] = sex
<ipython-input-36-80506ea1d8d5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['연령대'] = age


In [ ]:

x_train = dataset[['고객번호', '성별', '연령대', '거주지역', '분기별_구매금액_평균', '분기별_영수증갯수_평균']]
x_train.거주지역 = np.nan_to_num(x_train.거주지역,nan = 0)
x_train.거주지역 = x_train.거주지역.astype("int")
y_train = dataset['label']
dataset.label.value_counts()

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


1    19305
0       78
Name: label, dtype: int64

In [ ]:
test = dataset.copy()

In [ ]:
test['분기별_구매금액_평균']=df_cust[['14_1분기_구매금액',
       '14_2분기_구매금액', '14_3분기_구매금액', '14_4분기_구매금액', '15_1분기_구매금액',
       '15_2분기_구매금액','15_3분기_구매금액']].T.mean()
test['분기별_영수증갯수_평균']=df_cust[['14년도_1분기_영수증갯수', '14년도_2분기_영수증갯수',
       '14년도_3분기_영수증갯수', '14년도_4분기_영수증갯수', '15년도_1분기_영수증갯수', '15년도_2분기_영수증갯수','15년도_3분기_영수증갯수']].T.mean()

In [ ]:
test['label'] = np.where(df_cust['15_4분기_구매금액'] != 0,1,0)

In [ ]:
test.거주지역 = np.nan_to_num(test.거주지역,nan = 0)
test.거주지역 = test.거주지역.astype('int')
x_test = test[['고객번호', '성별', '연령대', '거주지역', '분기별_구매금액_평균', '분기별_영수증갯수_평균']]
y_test = test['label']


In [ ]:
test.label.value_counts()

1    19319
0       64
Name: label, dtype: int64

In [ ]:
lr = LogisticRegression()
lr.fit(x_train,y_train)
preds=lr.predict(x_test)
f1_score(preds,y_test)

0.9983463386905069

In [ ]:
p=list(preds)
y = y_test.values.tolist()
id = []
cnt = 0
for i,j in zip(p,y):
  cnt +=1
  if j != i:
    print(cnt)
    id.append(cnt)

In [ ]:
dataset.iloc[id]

,고객번호,성별,연령대,거주지역,분기별_구매금액_평균,분기별_영수증갯수_평균,label
174,175,0,9,55.0,4.120923e+06,56.000000,1
460,461,0,9,460.0,1.129647e+06,59.166667,1
905,906,0,8,100.0,1.614656e+06,66.666667,1
1171,1172,0,8,16.0,1.171768e+07,91.666667,1
1208,1209,0,8,24.0,3.386207e+06,99.000000,1
...,...,...,...,...,...,...,...
19225,19226,0,5,460.0,2.629698e+06,44.166667,1
19229,19230,1,8,60.0,1.702333e+04,0.500000,0
19230,19231,0,5,0.0,0.000000e+00,0.000000,1
19240,19241,0,5,100.0,1.114680e+06,85.500000,1


In [ ]:
goods_=goods_order.sort_values(['구매일자','구매시간'])

In [ ]:
goods_order[goods_order['고객번호'] == 1]['영수증번호']

## Reorder merge

https://drive.google.com/drive/folders/1_AqSMA_G8-YxtzKTjOmLaVuC-YtFtsIK?usp=share_link

https://drive.google.com/drive/folders/1-E1A4AHP_TrNqD4I7UUp5M52Jd4IQALi?usp=sharing

In [ ]:
d=p.merge_goods_order()

In [ ]:
d['reorder'] = 0

In [ ]:
import os
li=os.listdir('/gdrive/MyDrive/reorder')

In [ ]:
df_list = []#
for i in li:
  df_list.append(pd.read_csv('/gdrive/MyDrive/reorder/'+i))
res1=pd.concat(df_list)

In [ ]:
len(res1)

28593030

In [ ]:
d['reorder']= res1.values.reshape(1,-1)[0]

In [ ]:
a=d.groupby(['소분류명','소분류코드'])['reorder'].count().reset_index(name = 'whole_cnt')

In [ ]:
b=d.groupby(['소분류명','소분류코드'])['reorder'].sum().reset_index(name ='cnt')

In [ ]:
df_goods=p.show_goods()

In [ ]:
a['reorder_rate']= b['cnt'] / a['whole_cnt']

In [ ]:
df_goods2=pd.merge(a[['소분류명','소분류코드','reorder_rate']],df_goods,on = ['소분류명','소분류코드'])

In [ ]:
df_goods3 = pd.merge(b,df_goods2,on = ['소분류명','소분류코드'])

In [ ]:
df_goods3.to_csv('/gdrive/MyDrive/goods.csv',index = False)

In [ ]:
@jit(nopython = False)
def get_reorder_rate_cust():
  res = []
  for i in p.get_customer_id_list():
    temp=d[d['고객번호'] == i]['reorder'].value_counts()
    if len(temp) == 2:
      res.append(temp.values[1]/temp.sum())
    else:
      res.append(0)
  return res
res = get_reorder_rate_cust()

In [ ]:
cust = p.show_customer()
cust['상품재구매비율'] = res

In [ ]:
cust.to_csv('/gdrive/MyDrive/cust.csv',index = False)

In [ ]:
d = p.train()

In [ ]:
d = p.merge_goods_order()

In [ ]:
a=d.groupby(['소분류명'])['reorder'].count().reset_index(name = 'whole_cnt')
b=d.groupby(['소분류명'])['reorder'].sum().reset_index(name ='cnt')
a['reorder_rate']= b['cnt'] / a['whole_cnt']
b['reorder_rate']= b['cnt'] / a['whole_cnt']

In [ ]:
df_goods = p.show_goods()
df_goods2 = pd.merge(b,df_goods,right_index = True,left_index = True,how = 'right')

In [ ]:
d = p.train()
@jit(nopython = False)
def get_reorder_rate_cust():
  res = []
  for i in p.get_customer_id_list():
    temp=d[d['고객번호'] == i]['reorder'].value_counts()
    if len(temp) == 2:
      res.append(temp.values[1]/temp.sum())
    else:
      res.append(0)
  return res
res = get_reorder_rate_cust()

In [ ]:
cust=p.show_customer()
cust['reorder_train'] = res

In [ ]:
cust.to_csv('cust+train_reorder.csv',index = False)

#train,test dataset

In [ ]:
li = os.listdir('/gdrive/MyDrive/프로젝트2/train_test')
path = '/gdrive/MyDrive/프로젝트2/train_test/'

In [ ]:
li

['train.csv',
 'test.csv',
 '상품_test.csv',
 '고객_test.csv',
 '고객_train.csv',
 '상품_train.csv',
 'cust+train_reorder (1).csv',
 '.ipynb_checkpoints',
 'order_cust.csv']

In [ ]:
df_list=[]
for i in li[2:-3]:
  df_list.append(pd.read_csv(path+i))

In [ ]:
len(df_list)

4

In [ ]:
goods_train=df_list[3]
cust_train=df_list[2]

In [ ]:
goods_test = df_list[0]
cust_test = df_list[1]

#cust_train

In [ ]:
d = p.merge_goods_order()

In [ ]:
a=d.groupby(['소분류명'])['reorder'].count().reset_index(name = 'whole_cnt')
b=d.groupby(['소분류명'])['reorder'].sum().reset_index(name ='cnt')
a['reorder_rate']= b['cnt'] / a['whole_cnt']
b['reorder_rate']= b['cnt'] / a['whole_cnt']

In [ ]:
cust = pd.read_csv('/gdrive/MyDrive/프로젝트2/train_test/cust+train_reorder (1).csv')
cust_train['reorder']=cust['reorder_train']

In [ ]:
b['소분류코드'] = goods_train['소분류코드']

In [ ]:
goods_train['cnt'] = b['cnt']
goods_train['reorder_rate'] = b['reorder_rate']

In [ ]:
cust_train=cust_train.rename({'reorder' : 'reorder_rate'},axis = 'columns')

In [ ]:
goods_train['소분류명'] = b['소분류명']

In [ ]:
d['eval_set'] = np.where(d.구매일자 < 20151000,'train','test')

In [ ]:
order=d[['영수증번호','고객번호','구매시간','소분류명','reorder','eval_set']]

In [ ]:
order_cust=pd.merge(order,cust_train,on = '고객번호')

In [ ]:
goods_train=goods_train.rename({'reorder_rate' : 'goods_reorder_rate'},axis =1)

In [ ]:
order_cust.to_csv('/gdrive/MyDrive/프로젝트2/train_test/order_cust.csv',index = False)

In [ ]:
pd.merge(order_cust,goods_train)

In [ ]:
goods_train.to_csv('/gdrive/MyDrive/프로젝트2/train_test/goods_train.csv',index = False)

#모두합치기

In [ ]:
gp=pd.read_csv('/gdrive/MyDrive/프로젝트2/train_test/final.csv',chunksize = 2000000)
gp_li = list(gp)
order_cust = pd.concat(gp_li)


In [ ]:
train=order_cust[order_cust['eval_set']== 'train']

In [ ]:
train= train[['영수증번호', '고객번호','reorder','구매상품수',
       '첫-마지막', '구매간격', '구매건수', '구매항목수', '구매_평균항목수', 'reorder_rate', '소분류코드',
       '상품당_총구매수', '처음_두번째주문비율', '각상품먼저구매고객수', '전체판매량/첫번째구매', '평균구매량', 'cnt',
       'goods_reorder_rate']]
train_target = train['reorder']
train.drop('reorder',axis = 1,inplace = True)

In [ ]:
final_train = pd.merge(train,pd.read_csv('/gdrive/MyDrive/프로젝트2/train_test/고객X상품_train.csv'),on = ['고객번호','소분류코드'])

In [ ]:
test = order_cust[order_cust['eval_set']== 'test']
test_target = test['reorder']

In [ ]:
test= test[['영수증번호', '고객번호','구매상품수',
       '첫-마지막', '구매간격', '구매건수', '구매항목수', '구매_평균항목수', 'reorder_rate', '소분류코드',
       '상품당_총구매수', '처음_두번째주문비율', '각상품먼저구매고객수', '전체판매량/첫번째구매', '평균구매량', 'cnt',
       'goods_reorder_rate']]
final_test = pd.merge(test,pd.read_csv('/gdrive/MyDrive/프로젝트2/train_test/고객X상품_test.csv'),on = ['고객번호','소분류코드'])

In [ ]:
final_train.to_csv('/gdrive/MyDrive/프로젝트2/train_test/final_train.csv',index = False)
final_test.to_csv('/gdrive/MyDrive/프로젝트2/train_test/final_test.csv',index = False)
test_target.to_csv('/gdrive/MyDrive/프로젝트2/train_test/final_train_target.csv',index = False)

In [ ]:
train_target.to_csv('/gdrive/MyDrive/프로젝트2/train_test/final_test_target.csv',index = False)

## 모델링

In [ ]:
gp=pd.read_csv('/gdrive/MyDrive/프로젝트2/train_test/final_train.csv',chunksize = 2000000)
gp_li = list(gp)
final_train = pd.concat(gp_li)

final_train[final_train.select_dtypes('float64').columns] = final_train.select_dtypes('float64').astype('float32')
final_train[final_train.select_dtypes('int64').columns] = final_train.select_dtypes('int64').astype('int32')


In [ ]:
train_target = pd.read_csv('/gdrive/MyDrive/프로젝트2/train_test/final_train_target.csv')
train_target[train_target.select_dtypes('int64').columns] = train_target.select_dtypes('int64').astype('int32')

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_val,y_train,y_val = train_test_split(final_train,train_target,train_size = 0.8,shuffle=False)

In [ ]:
gp=pd.read_csv('/gdrive/MyDrive/프로젝트2/train_test/final_test.csv',chunksize = 2000000)
gp_li = list(gp)
final_test = pd.concat(gp_li)
test_target = pd.read_csv('/gdrive/MyDrive/프로젝트2/train_test/final_test_target.csv')
final_test[final_test.select_dtypes('float64').columns] = final_test.select_dtypes('float64').astype('float32')
final_test[final_test.select_dtypes('int64').columns] = final_test.select_dtypes('int64').astype('int32')
test_target[test_target.select_dtypes('int64').columns] = test_target.select_dtypes('int64').astype('int32')

In [ ]:
final_test.isna()

,영수증번호,고객번호,구매상품수,첫-마지막,구매간격,구매건수,구매항목수,구매_평균항목수,reorder_rate,소분류코드,...,전체판매량/첫번째구매,평균구매량,cnt,goods_reorder_rate,특정상품구매횟수,특정상품포함비율,처음구매횟수,마지막구매횟수,전체구매횟수/특정상품포함,특정상품구매후_특정상품구매비율
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3451755,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3451756,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3451757,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3451758,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
res = []
model = LGBMClassifier(num_leaves = 15, learning_rate = 0.5,histogram_pool_size=16384)
#models = [LGBMClassifier(histogram_pool_size=16384),DecisionTreeClassifier(),LogisticRegression()]
#for model in models:
model.fit(x_train.fillna(0),y_train.fillna(0),eval_set = (x_val,y_val), eval_metric = 'f1')
preds = model.predict(final_test.fillna(0))
res.append({'accuracy' : accuracy_score(preds,test_target),'recall' : recall_score(preds,test_target),'precision' : precision_score(preds,test_target),'f1' : f1_score(preds,test_target),'roc_auc' : roc_auc_score(preds,test_target)})
#models[1].fit(x_train.fillna(0),y_train.fillna(0))
#preds = models[1].predict(final_test)
#res.append(f1_score(preds,test_target))
res

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1]	valid_0's binary_logloss: 0.189795
[2]	valid_0's binary_logloss: 0.18193
[3]	valid_0's binary_logloss: 0.178553
[4]	valid_0's binary_logloss: 0.175817
[5]	valid_0's binary_logloss: 0.174331
[6]	valid_0's binary_logloss: 0.173445
[7]	valid_0's binary_logloss: 0.172673
[8]	valid_0's binary_logloss: 0.171834
[9]	valid_0's binary_logloss: 0.171128
[10]	valid_0's binary_logloss: 0.170905
[11]	valid_0's binary_logloss: 0.170588
[12]	valid_0's binary_logloss: 0.170358
[13]	valid_0's binary_logloss: 0.169871
[14]	valid_0's binary_logloss: 0.169704
[15]	valid_0's binary_logloss: 0.169394
[16]	valid_0's binary_logloss: 0.169086
[17]	valid_0's binary_logloss: 0.168811
[18]	valid_0's binary_logloss: 0.16883
[19]	valid_0's binary_logloss: 0.168681
[20]	valid_0's binary_logloss: 0.168643
[21]	valid_0's binary_logloss: 0.168531
[22]	valid_0's binary_logloss: 0.16844
[23]	valid_0's binary_logloss: 0.168317
[24]	valid_0's binary_logloss: 0.168368
[25]	valid_0's binary_logloss: 0.168297
[26]	valid_0

[{'accuracy': 0.9133653556446566,
  'recall': 0.34868084484396694,
  'precision': 0.3547289466203066,
  'f1': 0.35167889414988635,
  'roc_auc': 0.6514247388049778}]

In [ ]:
res#lgbm Dt Logistic

# 모델링2: automl(LightGBM, Decision Tree)

##!pip install automl

In [ ]:
!pip install mljar-supervised

validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }

In [ ]:
from supervised.automl import AutoML
automl = AutoML(results_path = '/gdrive/MyDrive/prediction2',algorithms=["LightGBM",'Decision Tree'], stack_models=False,mode = 'Perform', eval_metric='f1', ml_task='binary_classification',n_jobs=-1)
#automl.set_params(lgbm = {'histogram_pool_size' :16384})
lgbm_bin_params = {
    "objective": ["binary"],
    "metric": ["binary_logloss"],
    "num_leaves": [3, 7, 15, 31],  
    "learning_rate": [0.05, 0.075, 0.1, 0.15,0.5],
    "feature_fraction": [0.8, 0.9, 1.0],
    "bagging_fraction": [0.8, 0.9, 1.0],
    "min_data_in_leaf": [5, 10, 15, 20, 30, 50],
    'histogram_pool_size' : [16384]
}

In [ ]:
dataset = pd.concat([final_train,final_test],axis = 0)
target = pd.concat([train_target,test_target],axis = 0)

In [ ]:
automl.fit(dataset,target)

# 모델링3: knn basic


In [ ]:
cluster = pd.read_csv('/gdrive/MyDrive/프로젝트2/train_test/고객군집f.csv')
cust_id=cluster[cluster['cluster2']==1]['고객번호'].values

In [ ]:
import surprise
gp = pd.read_csv('/gdrive/MyDrive/프로젝트2/구매상품TR_reorder.csv',chunksize = 2000000)
gp_li = list(gp)
df=pd.concat(gp_li)

reorder_rate=df.groupby(['고객번호','소분류명'])[['reorder']].sum()
total_ordered_goods=df.groupby(['고객번호','소분류명'])[['reorder']].count().reset_index()
o=reorder_rate.reset_index()
o['rate'] = o['reorder']/total_ordered_goods['reorder']


en = LabelEncoder()
goods_name=en.fit_transform(o['소분류명'])

o['소분류명_encoded'] = goods_name
bins =pd.cut(o['rate'],5,labels = [1,2,3,4,5])
o['ratings'] = bins

o.columns = ['user_id','소분류명','reorder','rate','item_id','raiting']
o.info()
p = o[['user_id','item_id','raiting']].astype('int16')
p=p.query('user_id in @cust_id')
reader = surprise.dataset.Reader(rating_scale = (0.5,5.0))
data=surprise.dataset.Dataset.load_from_df(p,reader)
trainset = data.build_full_trainset()

sim_options = {'name': 'cosine'}
algo = surprise.KNNBasic(sim_options=sim_options)
try :
  algo.fit(trainset)
except:
  print(0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6134917 entries, 0 to 6134916
Data columns (total 6 columns):
 #   Column   Dtype   
---  ------   -----   
 0   user_id  int64   
 1   소분류명     object  
 2   reorder  int64   
 3   rate     float64 
 4   item_id  int64   
 5   raiting  category
dtypes: category(1), float64(1), int64(3), object(1)
memory usage: 239.9+ MB
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
len(p.item_id.unique())

3415

In [ ]:
b=algo.predict(uid = 288,iid = 111)

In [ ]:
@jit (nopython = False)
def x(uid,iid):
  user_id = []
  item_id = []
  score = []
  for i in uid:
    for j in iid:
      a = algo.predict(i,j)
      user_id.append(a[0])
      item_id.append(a[1])
      score.append(a[3])
    print(i)
  df_rating=pd.DataFrame(user_id,columns = ['user_id'])
  df_rating['item_id'] = item_id
  df_rating['score'] = score
  return df_rating

In [ ]:
res=x(cust_id,p.item_id.unique())

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
5091
5092
5093
5095
5098
5099
5100
5103
5105
5108
5109
5110
5111
5115
5117
5119
5122
5123
5126
5127
5128
5130
5132
5135
5138
5140
5144
5145
5146
5152
5153
5154
5156
5157
5160
5165
5166
5172
5173
5176
5179
5180
5183
5185
5187
5189
5191
5192
5193
5197
5200
5201
5206
5209
5217
5220
5223
5228
5229
5234
5239
5240
5242
5246
5253
5255
5260
5261
5264
5277
5278
5279
5280
5282
5283
5286
5287
5289
5292
5296
5298
5303
5307
5311
5312
5314
5317
5320
5322
5323
5330
5332
5335
5336
5339
5341
5346
5347
5348
5353
5354
5356
5359
5367
5375
5376
5378
5379
5383
5387
5390
5391
5395
5397
5398
5400
5403
5405
5406
5407
5414
5417
5418
5421
5423
5424
5425
5428
5430
5433
5438
5441
5444
5450
5451
5458
5468
5469
5471
5473
5474
5480
5481
5487
5488
5492
5495
5497
5500
5502
5505
5506
5507
5510
5517
5523
5527
5530
5533
5535
5537
5541
5543
5549
5550
5551
5553
5559
5561
5565
5571
5572
5573
5577
5581
5585
5586
5591
5593
5600
5609
5611
5616
5618
5619
5633
5635
5636
5638
5640
5641
5642
5645

In [ ]:
res.to_csv('/gdrive/MyDrive/프로젝트2/rating.csv',index = False)

In [ ]:
df_list = []
for i in res.user_id.unique():
  df_list.append(res[res['user_id'] == i].sort_values('score',ascending = False)[:5])


In [ ]:
temp=pd.concat(df_list,axis = 0)

In [ ]:
temp=temp.reset_index()

In [ ]:
temp.drop('index',axis = 1,inplace = True)

In [ ]:
pd.merge(temp,o,left_index =True, right_index = True).to_csv('top5knn+item_name.csv',index = False)

#기대효과

In [ ]:
cust = pd.read_csv('/gdrive/MyDrive/프로젝트2/train_test/고객군집f.csv')

In [ ]:
cust.drop('cluster2',axis = 1,inplace = True)

In [ ]:
cust2=cust.iloc[:,:-1]

In [ ]:
cust2[]

,고객번호,성별,연령대,거주지역,구매상품수,첫-마지막,구매간격,구매건수,구매항목수,구매_평균항목수,재구매율,영수증평균개수,구매금액평균,rfm,총구매금액,총영수증개수,최다구매물품,cluster
0,1,1,0,60.0,1057,721,3.121212,681,255,1.552129,0.082308,85.125,1.063032e+07,111,85042550,681,2842,2
1,2,1,0,100.0,1116,726,2.547368,676,140,1.650888,0.106631,84.500,1.248510e+07,111,99880760,676,2349,2
2,3,1,0,33.0,1191,724,1.832911,490,161,2.430612,0.280437,61.250,3.956459e+05,100,3165167,490,2500,2
3,4,2,0,16.0,930,727,2.806950,533,254,1.744841,0.027957,66.625,2.086726e+06,100,16693810,533,2842,2
4,5,1,0,100.0,719,725,3.072034,426,80,1.687793,0.123783,53.250,1.654634e+06,100,13237070,426,2411,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19378,19379,2,8,69.0,584,461,2.114679,366,133,1.595628,0.095890,45.750,7.224100e+05,100,5779280,366,3196,2
19379,19380,2,8,39.0,440,385,2.655172,232,107,1.896552,0.050000,29.000,4.891221e+05,100,3912977,232,3046,2
19380,19381,2,4,460.0,805,363,1.824121,402,281,2.002488,0.029814,50.250,7.550836e+05,100,6040669,402,929,1
19381,19382,2,3,460.0,906,344,1.694581,435,307,2.082759,0.034216,54.375,5.502216e+05,100,4401773,435,1429,1


In [ ]:
y_train=cust2['총구매금액']
x_train = cust2.drop('총구매금액',axis = 1)

In [ ]:
x_test=x_train.copy()

In [ ]:
x_train['재구매율'].max()

0.5

In [ ]:
np.arange(1.1,2.0,0.1)

array([1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9])

In [ ]:
from lightgbm import LGBMRegressor

models = LGBMRegressor()
model.fit(x_train,y_train)
res  =  []
for i in np.arange(0.1,,0.1):
  x_test['재구매율']=x_train['재구매율'] +i
  preds=model.predict(x_test)
  res.append((preds.sum()-y_train.sum()))


In [ ]:
cust2.corr()['재구매율']

고객번호        0.017007
성별         -0.089148
연령대         0.029680
거주지역       -0.023597
구매상품수      -0.221190
첫-마지막      -0.085984
구매간격        0.044030
구매건수       -0.051195
구매항목수      -0.485025
구매_평균항목수   -0.224747
재구매율        1.000000
영수증평균개수    -0.051195
구매금액평균      0.255237
rfm        -0.014626
총구매금액       0.255237
총영수증개수     -0.051195
최다구매물품     -0.033803
Name: 재구매율, dtype: float64

84741724678.45528

114330060.83027649